# Erstellen einer FHIR Observation Ressource mit dem Python fhirclient zur Angabe der Herzfrequenz
Um den Python fhirclient zu installieren kann man entweder die aktuellste Version aus dem [GitHub Repository](https://github.com/smart-on-fhir/client-py) nehmen oder die zuletzt auf [PyPi](https://pypi.org/project/fhirclient/) veröffentlichte Version. Da diese jedoch noch STU-3 ist, nutzen wir für die folgenden Beispiele die GitHub Version (R4).

In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

In [ ]:
from IPython.display import IFrame
import json
from fhirclient.models import (
    patient,
    observation
)

In [ ]:
# Instanziieren einer Observation
my_observation = observation.Observation()

## Mit der Observation Ressource vertraut machen
Um erstmal einen Überblick davon zu erhalten wie die Observation Ressource aufgebaut ist, bzw. welche Attribute diese enthält kann die Funktion **elementProperties()** aufgerufen werden, diese gibt dann alle Attribute der fhirclient Klasse **Observation()** mit ihren jeweiligen Eigenschaften zurück.

In [ ]:
for property in my_observation.elementProperties():
    print(property)

## Hinzufügen von Metadaten über die Ressource
Zu den Metadaten gehört u.A. das sogenannte FHIR Profil. In diesem Fall nutzen wir das Herzfrequenz Profil aus der FHIR Spezifikation.

In [ ]:
IFrame('https://hl7.org/fhir/heartrate.html#10.1.25.2', width=1200, height=630)

Jede FHIR Ressource hat Metadaten

In [ ]:
IFrame('https://www.hl7.org/fhir/resource.html#resource', width=1200, height=330)

In [ ]:
from fhirclient.models import meta

In [ ]:
obs_meta = meta.Meta()
obs_meta.profile = ['http://hl7.org/fhir/StructureDefinition/heartrate']
my_observation.meta = obs_meta

## Festlegen des Status dieser Observation

In [ ]:
status = 'final'
my_observation.status = status

## Festlegen einer Kategorie

In [ ]:
IFrame('https://hl7.org/fhir/datatypes.html#codeableconcept', width=1200, height=330)

In [ ]:
from fhirclient.models import (
    codeableconcept,
    coding
)

In [ ]:
IFrame('http://terminology.hl7.org/CodeSystem/observation-category', width=1200, height=330)

In [ ]:
category = codeableconcept.CodeableConcept()
category_coding = coding.Coding()
category_coding.system = 'http://terminology.hl7.org/CodeSystem/observation-category'
category_coding.code = 'vital-signs'
category.coding = [category_coding]
my_observation.category = [category]

In [ ]:
print(json.dumps(category.as_json(), indent=4))

## Vergeben eines Codes aus einer Terminologie, der die Bedeutung 'Herzfrequenz' repräsentiert

In [ ]:
code = codeableconcept.CodeableConcept()
code_coding = coding.Coding()
code_coding.system = 'http://loinc.org'
code_coding.code = '8867-4'
code_coding.display = 'Heart rate'
code.coding = [code_coding]
my_observation.code = code

In [ ]:
print(json.dumps(code.as_json(), indent=4))

## Festlegen einer Referenz auf die zuvor erstellte Patienten-Ressource

In [ ]:
from fhirclient.models import fhirreference
import requests

### GET zuvor erstellte Patienten-Ressource

In [ ]:
fhir_test_server = 'https://server.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

response = requests.get(url=f'{fhir_test_server}/Patient?family=Bach', headers=headers)

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

### ID des zuvor erstellten Patientens holen

In [ ]:
# Response Text als Python Dictionary holen
response_dict = json.loads(response.text)
response_dict

In [ ]:
response_dict['entry'][0]['resource']['id']

In [ ]:
subject = fhirreference.FHIRReference()
pat_id = response_dict['entry'][0]['resource']['id']
subject.reference = f'Patient/{pat_id}'
my_observation.subject = subject

In [ ]:
print(json.dumps(subject.as_json(), indent=4))

## Angeben des Zeitpunktes, wann die Observation gemacht wurde

In [ ]:
from fhirclient.models import fhirdate

In [ ]:
date = fhirdate.FHIRDate('2020-11-24')
my_observation.effectiveDateTime = date

In [ ]:
print(json.dumps(date.as_json(), indent=4))

## Angabe des gemessenen Wertes

In [ ]:
from fhirclient.models import quantity

In [ ]:
measured_quantity = quantity.Quantity()
measured_quantity.value = 60
measured_quantity.unit = 'beats per minute'
measured_quantity.code = '/min'
measured_quantity.system = 'http://unitsofmeasure.org'
my_observation.valueQuantity = measured_quantity

In [ ]:
print(json.dumps(measured_quantity.as_json(), indent=4))

## Die Observation-Ressource validieren und an den FHIR Testserver senden

In [ ]:
fhir_test_server = 'https://server.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [ ]:
response = requests.post(f'{fhir_test_server}/Observation/$validate', headers = headers, data = json.dumps(my_observation.as_json()))
data = response.json()

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
print(json.dumps(data, indent=4))

In [ ]:
response = requests.post(f'{fhir_test_server}/Observation', headers = headers, data = json.dumps(my_observation.as_json()))
data = response.json()

In [ ]:
IFrame(f'http://http.cat/{response.status_code}', width=750, height=600)

In [ ]:
print(json.dumps(data, indent=4))